In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import LabelEncoder

2024-02-24 13:25:36.017704: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def my_custom_operation(inputs):
    squared_inputs = tf.square(inputs)
    concatenated = tf.concat([inputs, squared_inputs], axis=-1)
    reshaped_output = tf.reshape(concatenated, [-1, 4, 1])
    return reshaped_output

In [3]:
inputs = tf.keras.Input(shape=(2,1))  # Adjust the shape according to your data.
custom_op_output = my_custom_operation(inputs)
flattened_output = tf.keras.layers.Flatten()(custom_op_output)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flattened_output)    
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss = "binary_crossentropy")

In [4]:
weights = np.array([[-0.3806], [0.3034], [-0.2139], [0.1545]])
bias = np.array([0]).reshape(1, )
bias_and_weights = []
bias_and_weights.append(weights)
bias_and_weights.append(bias)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2, 1)]               0         []                            
                                                                                                  
 tf.math.square (TFOpLambda  (None, 2, 1)                 0         ['input_1[0][0]']             
 )                                                                                                
                                                                                                  
 tf.concat (TFOpLambda)      (None, 2, 2)                 0         ['input_1[0][0]',             
                                                                     'tf.math.square[0][0]']      
                                                                                              

In [5]:
all_data = pd.read_csv("spline-test.csv", index_col=2)
ct_data = all_data['CT'].values #drop the /N values instead
age_data = all_data['PatientAge'].values
print(ct_data)

[23.95 33.65 27.28 31.22 26.8  17.34 31.08 29.66 24.8  32.4  23.6  28.4 ]


In [6]:
X = np.column_stack((age_data, ct_data))
X

array([[15.        , 23.95      ],
       [17.        , 33.65      ],
       [ 8.        , 27.28      ],
       [ 7.        , 31.22      ],
       [ 0.66666667, 26.8       ],
       [ 6.        , 17.34      ],
       [ 8.        , 31.08      ],
       [15.        , 29.66      ],
       [ 5.        , 24.8       ],
       [10.        , 32.4       ],
       [10.        , 23.6       ],
       [ 6.        , 28.4       ]])

In [7]:
Y = model.predict(X)
Y

1/1 [==============================] - 0s 53ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)